### Import required library and packages

In [2]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
import matplotlib.pyplot as plt
from tqdm import tqdm
import urllib
import IPython

2023-04-28 16:31:15.661357: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 16:31:15.784558: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## Build UNet Architecture

In [6]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

In [7]:
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [8]:
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding='same')(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [9]:
def build_unet(input_shape):
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    b1 = conv_block(p4, 1024)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    """ Output """
    outputs = Conv2D(1, (1, 1), padding="same", activation="sigmoid")(d4)
    return Model(inputs, outputs, name="U-Net")

In [12]:
def load_data(dataset_path):
    images = sorted(glob(os.path.join(dataset_path, "images/*0.JPG")))
    masks = sorted(glob(os.path.join(dataset_path, "masks/*")))
    train_x, test_x = train_test_split(images, test_size=0.2, random_state=42)
    train_y, test_y = train_test_split(masks, test_size=0.2, random_state=42)
    return (train_x, train_y), (test_x, test_y)

In [15]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    for i in range(1,6):
        tmp_channel = cv2.imread("".join([path[:-5],f'{i}.TIF']), cv2.IMREAD_GRAYSCALE)
        x = cv2.merge((x,tmp_channel))
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    # (256, 256, 8)
    return x

In [14]:
def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [13]:
def preprocess(image_path, mask_path):
    def f(image_path, mask_path):
        image_path = image_path.decode()
        mask_path = mask_path.decode()
        x = read_image(image_path)
        y = read_mask(mask_path)
        return x, y
    image, mask = tf.numpy_function(f, [image_path, mask_path], [tf.float32, tf.float32])
    image.set_shape([256, 256, 8])
    mask.set_shape([256, 256, 1])
    return image, mask

In [16]:
def tf_dataset(images, masks, batch=8):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

In [11]:
#mirrored_strategy = tf.distribute.MirroredStrategy()

2023-03-27 16:49:04.419607: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 16:49:05.520265: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78971 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:31:00.0, compute capability: 8.0
2023-03-27 16:49:05.521478: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 78971 MB memory:  -> device: 1, name: NVIDIA A100 80GB PCIe, pci bus id: 0000:98:00.0, compute capability: 8.0


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


### Train the model

In [17]:
""" Hyperparameters """
dataset_path = "multispectral_dataset"
input_shape = (256, 256, 8)
batch_size = 8
epochs = 10
lr = 1e-4
model_path = "saved_models/unet.h5"
csv_path = "data.csv"

""" Loading the dataset """
(train_x, train_y), (test_x, test_y) = load_data(dataset_path)

train_dataset = tf_dataset(train_x, train_y, batch=batch_size)
valid_dataset = tf_dataset(test_x, test_y, batch=batch_size)

""" Model """
#with mirrored_strategy.scope():
model = build_unet(input_shape)
model.compile(
    loss="binary_crossentropy",
    optimizer=tf.keras.optimizers.Adam(lr),
    metrics=[
        tf.keras.metrics.MeanIoU(num_classes=2),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision()
    ]
)

callbacks = [
    ModelCheckpoint(model_path, monitor="val_loss", verbose=1),
    ReduceLROnPlateau(monitor="val_loss", patience=5, factor=0.1, verbose=1),
    CSVLogger(csv_path),
    EarlyStopping(monitor="val_loss", patience=10)
]

train_steps = len(train_x)//batch_size
if len(train_x) % batch_size != 0:
    train_steps += 1
    
valid_steps = len(test_x)//batch_size
if len(test_x) % batch_size != 0:
    valid_steps += 1

model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs = epochs,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

2023-04-28 16:12:04.500767: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 16:12:04.670643: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1637] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18026 MB memory:  -> device: 0, name: NVIDIA A100 80GB PCIe MIG 2g.20gb, pci bus id: 0000:98:00.0, compute capability: 8.0


Epoch 1/10


2023-04-28 16:12:09.878133: I tensorflow/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700


53/53 [==============================] - ETA: 0s - loss: 0.5706 - mean_io_u: 0.1728 - recall: 0.7991 - precision: 0.7608
Epoch 1: saving model to saved_models/unet.h5
53/53 [==============================] - 93s 2s/step - loss: 0.5706 - mean_io_u: 0.1728 - recall: 0.7991 - precision: 0.7608 - val_loss: 0.7184 - val_mean_io_u: 0.1839 - val_recall: 0.0014 - val_precision: 0.1824 - lr: 1.0000e-04
Epoch 2/10
53/53 [==============================] - ETA: 0s - loss: 0.4690 - mean_io_u: 0.1728 - recall: 0.8666 - precision: 0.8140
Epoch 2: saving model to saved_models/unet.h5
53/53 [==============================] - 61s 1s/step - loss: 0.4690 - mean_io_u: 0.1728 - recall: 0.8666 - precision: 0.8140 - val_loss: 0.8740 - val_mean_io_u: 0.1839 - val_recall: 1.8696e-04 - val_precision: 0.7363 - lr: 1.0000e-04
Epoch 3/10
53/53 [==============================] - ETA: 0s - loss: 0.4316 - mean_io_u: 0.1728 - recall: 0.8622 - precision: 0.8369
Epoch 3: saving model to saved_models/unet.h5
53/53 [======

### Test the model

In [16]:
# https://raw.githubusercontent.com/nikhilroxtomar/Unet-for-Person-Segmentation/main/images/Black-Widow-Avengers.jpg

#test_images = glob("images/*")


test_images = [
    'https://raw.githubusercontent.com/nikhilroxtomar/Unet-for-Person-Segmentation/main/images/Black-Widow-Avengers.jpg'
]

model = tf.keras.models.load_model("saved_models/unet.h5")
for path in tqdm(test_images, total=len(test_images)):
    
    req = urllib.request.urlopen(path)
    imgarr = np.asarray(bytearray(req.read()), dtype=np.uint8)
    
    x = cv2.imdecode(imgarr, -1)
    
    #x = cv2.imread(path, cv2.IMREAD_COLOR)
    original_image = x
    h, w, _ = x.shape
    
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)

    pred_mask = model.predict(x)[0]
    pred_mask = cv2.resize(pred_mask, (w, h))
    pred_mask = np.expand_dims(pred_mask, axis=-1)
    pred_mask = pred_mask > 0.5
    
    background_mask = np.abs(1- pred_mask)
        
    masked_image = original_image * pred_mask
    
    background_mask = np.concatenate([background_mask, background_mask, background_mask], axis=-1)
    background_mask = background_mask * [0, 0, 0]
    
    masked_image = masked_image + background_mask
    name = path.split("/")[-1]
    cv2.imwrite(f"{name}.png", masked_image)

  0%|                                                     | 0/1 [00:00<?, ?it/s]

1/1 [==============================] - 1s 879ms/step


100%|█████████████████████████████████████████████| 1/1 [00:01<00:00,  1.72s/it]


In [4]:
#model = tf.keras.models.load_model("saved_models/unet.h5")
path = 'multispectral_dataset/images/split9_0010.JPG'
x = cv2.imread(path, cv2.IMREAD_COLOR)
original_image = x
for i in range(1,6):
        tmp_channel = cv2.imread("".join([path[:-5],f'{i}.TIF']), cv2.IMREAD_GRAYSCALE)
        x = cv2.merge((x,tmp_channel))
h, w, _ = x.shape

x = cv2.resize(x, (256, 256))
x = x/255.0
x = x.astype(np.float32)
x = np.expand_dims(x, axis=0)

pred_mask = model.predict(x)[0]
pred_mask = cv2.resize(pred_mask, (w, h))
pred_mask = np.expand_dims(pred_mask, axis=-1)
pred_mask = pred_mask > 0.5
    
background_mask = np.abs(1- pred_mask)
        
masked_image = original_image * pred_mask
   
background_mask = np.concatenate([background_mask, background_mask, background_mask], axis=-1)
background_mask = background_mask * [0, 0, 0]
    
masked_image = masked_image + background_mask
cv2.imwrite("test.png", masked_image)

1/1 [==============================] - 0s 155ms/step


True